# Prueba de FFT en audio

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

%matplotlib

Using matplotlib backend: Qt5Agg


In [8]:
fs, y = wavfile.read('./data/56053__ldk1609__violin-tremolo-a-3.wav')

In [12]:
plt.plot(y)

In [13]:
fy = np.fft.fft(y)

In [18]:
frecs = np.linspace(0, 2*fs, len(y))
plt.plot(frecs, abs(fy))
plt.xlim(0,fs)

(0, 44100)

In [19]:
from scipy.signal import spectrogram

In [57]:
f, t, sxx = spectrogram(y, fs, nperseg=300)

In [60]:
plt.imshow(np.log(sxx), extent=(t[0], t[-1], f[0], f[-1]), aspect=0.00011, origin='lower', interpolation=None) 

In [48]:
f

array([0.00000e+00, 1.47000e+01, 2.94000e+01, ..., 2.20206e+04,
       2.20353e+04, 2.20500e+04])